In [11]:
import os

def display_contents(path, indent=0):
    for root, dirs, files in os.walk(path):
        current_indent = ' ' * indent
        print(f"{current_indent}{root}")
        
        for file in files[:3]:
            print(f"{current_indent}  File: {file}")
        if len(files) > 3:
            print(f"{current_indent}  ... [ total : {len(files)}]")
        
        for directory in dirs:
            display_contents(os.path.join(root, directory), indent + 2)

dataset_path = "..\dataset"  # put the path to the dataset
display_contents(dataset_path)


<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
C:\Users\di-co\AppData\Local\Temp\ipykernel_15136\261459490.py:16: SyntaxWarning: invalid escape sequence '\d'
  dataset_path = "..\dataset"  # put the path to the dataset


..\dataset
  File: compressed.zip
  File: labels.csv
  ..\dataset\traffic_Data
    ..\dataset\traffic_Data\DATA
      ..\dataset\traffic_Data\DATA\0
        File: 000_0001.png
        File: 000_0002.png
        File: 000_0003.png
        File: 000_0004.png
        File: 000_0005.png
        ... [ total : 118]
      ..\dataset\traffic_Data\DATA\1
        File: 001_0001.png
        File: 001_0002.png
        File: 001_0003.png
        File: 001_0004.png
        File: 001_0005.png
        ... [ total : 40]
      ..\dataset\traffic_Data\DATA\10
        File: 010_0001.png
        File: 010_0002.png
        File: 010_0003.png
        File: 010_0004.png
        File: 010_0005.png
        ... [ total : 70]
      ..\dataset\traffic_Data\DATA\11
        File: 011_0001.png
        File: 011_0002.png
        File: 011_0003.png
        File: 011_0004.png
        File: 011_0005.png
        ... [ total : 138]
      ..\dataset\traffic_Data\DATA\12
        File: 012_0001.png
        File: 012_0002.png
